In [375]:
import pandas as pd
import numpy as np
import spacy
import re

from lxml import html
from gensim.models import Word2Vec, KeyedVectors, FastText
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, classification_report
from scipy.spatial.distance import cosine


from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

## Определение функций

In [4]:
nlp = spacy.load('ru_core_news_sm')

In [54]:
def process(text):
    if type(text) == str:
        return [token.lemma_+'_'+token.pos_ for token in nlp(text) if token.pos_ not in ('AUX','PUNCT') \
                and token.text.lower() not in stops]
    return []

In [339]:
def vectorize_sents(sents, vectorizer, model):
    vocab = sorted(vectorizer.vocabulary_, key=lambda x: vectorizer.vocabulary_[x])
    word_vectors = np.array([model[word] if word in model else np.zeros(model.vectors.shape[1]) for word in vocab])
    return vectorizer.transform(sents) * word_vectors

# Загрузка данных

## Данные соревнования Paraphrases

In [157]:
corpus_xml = html.fromstring(open('paraphrases/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []
stops = set(stopwords.words('russian'))

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':[str(i) for i in texts_1], 'text_2':[str(i) for i in texts_2],
                     'label':[int(i) for i in classes]})

In [158]:
data.to_excel("task_data.xlsx")

In [21]:
tqdm.pandas()

In [191]:
data['text_1_norm'] = data['text_1'].progress_apply(process)
data['text_2_norm'] = data['text_2'].progress_apply(process)

100%|███████████████████████████████████████████████████████████████████████████████████| 7227/7227 [00:42<00:00, 171.32it/s]


In [23]:
data.head()

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,"[полицейский_NOUN, разрешить_VERB, стрелять_VE...","[полиция_NOUN, мочь_VERB, разрешить_VERB, стре..."
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,"[право_NOUN, полицейский_NOUN, проникновение_N...","[правило_NOUN, внесудебный_ADJ, проникновение_..."
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,"[президент_NOUN, египет_PROPN, ввести_VERB, чр...","[власть_NOUN, египет_PROPN, угрожать_VERB, вве..."
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,"[вернуться_VERB, сирия_PROPN, россиянин_NOUN, ...","[самолёт_NOUN, мчс_PROPN, вывезти_VERB, россия..."
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,"[москва_PROPN, сирия_PROPN, вернуться_VERB, 2_...","[самолёт_NOUN, мчс_PROPN, вывезти_VERB, россия..."


## Готовая модель Word2Vec

Используем модель http://vectors.nlpl.eu/repository/20/180.zip

In [24]:
ready_model = KeyedVectors.load_word2vec_format('gensim_model/model.bin',
                                               binary=True)

In [25]:
[i for i in ready_model.vocab][:5]

['так_ADV', 'быть_VERB', 'мочь_VERB', 'год_NOUN', 'человек_NOUN']

Формат данных в модели - lemma_POS

## Данные для обучения моделей gensim:

Используем данные проекта OpenCorpora (http://opencorpora.org/files/export/annot/annot.opcorpora.xml.zip) - морфологическую рзаметку использовать не будем, вместо этого анализируем их при помощи spacy:

In [27]:
with open('data/annot.opcorpora.xml','r',encoding='utf-8') as inp:
    xml = inp.read()

In [29]:
sentences = re.findall('<source>(.*)?</source>', xml)

In [31]:
len(sentences)

110304

In [41]:
sentences = pd.Series(sentences).progress_apply(process)

100%|███████████████████████████████████████████████████████████████████████████████| 110304/110304 [16:14<00:00, 113.19it/s]


## Обучение модели Word2vec

In [43]:
my_model = Word2Vec(sentences)

## Обучение модели FastText

In [311]:
my_model1 = FastText(sentences)

## Обучение TfIdfVectorizer'а

In [391]:
do_nothing = lambda x: x

In [392]:
all_texts = pd.concat([data['text_1_norm'], data['text_2_norm']])

In [393]:
tfidf_vec = TfidfVectorizer(preprocessor=do_nothing, tokenizer=do_nothing)

In [394]:
tfidf_vec.fit(all_texts)

c:\users\k1l77\desktop\182e~1\term_p~1\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(preprocessor=<function <lambda> at 0x000002C411F03598>,
                tokenizer=<function <lambda> at 0x000002C411F03598>)

## Подготовка данных

Векторизуем тексты при помощи моделей, усреднив значения используя TfIdf:

In [257]:
X_tfidf_my = np.concatenate((vectorize_sents(data['text_1_norm'], tfidf_vec, my_model.wv),
                         vectorize_sents(data['text_2_norm'], tfidf_vec, my_model.wv)), axis=1)

In [258]:
X_tfidf_my.shape

(7227, 200)

In [260]:
X_tfidf_ready = np.concatenate((vectorize_sents(data['text_1_norm'], tfidf_vec, ready_model),
                         vectorize_sents(data['text_2_norm'], tfidf_vec, ready_model)), axis=1)

In [261]:
X_tfidf_ready.shape

(7227, 600)

Сохраним целевую переменную как y:

In [279]:
y = data.label

# Задание 1

Будем использовать усреднённые при помоши TfIdf эмбеддинги

In [306]:
def f1_macro(clf, X, y):
    ## средний f1 по классам
    y_pred = clf.predict(X)
    return f1_score(y, y_pred, average='macro')

def f1_micro(clf, X, y):
    ## f1 относительно каждого объекта
    y_pred = clf.predict(X)
    return f1_score(y, y_pred, average='micro')
    
clf = LogisticRegression(solver='liblinear')

Проведём кросс-валидацию для обученной нами модели Word2Vec:

In [299]:
my_model_scores_macro = cross_val_score(clf, X_tfidf_my, y, scoring=f1_macro)
my_model_scores_micro = cross_val_score(clf, X_tfidf_my, y, scoring=f1_micro)

И для готовой модели:

In [301]:
ready_model_scores_macro = cross_val_score(clf, X_tfidf_ready, y, scoring=f1_macro)
ready_model_scores_micro = cross_val_score(clf, X_tfidf_ready, y, scoring=f1_micro)

Сравним средний f1-score для обученной и готовой моделей

In [305]:
print(f'''
Macro F1 (Pretrained) {np.mean(ready_model_scores_macro)}
Micro F1 (Pretrained) {np.mean(ready_model_scores_micro)}
Macro F1 (Trained)    {np.mean(my_model_scores_macro)}
Micro F1 (Trained)    {np.mean(my_model_scores_micro)}
''')


Macro F1 (Pretrained) 0.38130403589026246
Micro F1 (Pretrained) 0.41303316151942837
Macro F1 (Trained)    0.34647277494724743
Micro F1 (Trained)    0.44665422331978927



<b>Вывод</b>: Наша модель спрваляется лучше в целом, но хуже для классов по отдельности

## Задание 2

Определим функцию, которая получив данные и модели, выдаст датафрейм с нужными расстояниями:

In [365]:
def double_cosine(a,b):
    assert len(a) == len(b)
    
    length = len(a)
    outp = np.zeros(length)
    
    for i in range(length):
        outp[i] = cosine(a[i], b[i])
    
    return outp

def cosine_dist(data, func):
    return double_cosine(func(data['text_1_norm']),
                 func(data['text_2_norm']))

def get_dataset(data, wv_model1, wv_model2, ft_model,
                svd_dim=200, nmf_dim=50):
    do_nothing = lambda x: x
    all_texts = pd.concat((data['text_1_norm'], data['text_2_norm']))
    
    print("Learning TF-IDF...")
    
    vec = TfidfVectorizer(preprocessor=do_nothing, tokenizer=do_nothing)
    all_texts = vec.fit_transform(all_texts)
    
    print("Learning SVD...")
    
    svd = TruncatedSVD(svd_dim)
    svd.fit(all_texts)
    
    print("Learning NMF...")
    
    nmf = NMF(nmf_dim)
    nmf.fit(all_texts)
    
    print("Calculating SVD distance...")
    svd_dist = cosine_dist(data, lambda x: svd.transform(vec.transform(x)))
    print("Calculating NMF distance...")
    nmf_dist = cosine_dist(data, lambda x: nmf.transform(vec.transform(x)))
    print("Calculating Word2Vec distance1...")
    wv1_dist = cosine_dist(data, lambda x: vectorize_sents(x, vec, wv_model1))
    print("Calculating Word2Vec distance2...")
    wv2_dist = cosine_dist(data, lambda x: vectorize_sents(x, vec, wv_model2))
    print("Calculating FastText distance...")
    ft_dist = cosine_dist(data, lambda x: vectorize_sents(x, vec, ft_model))
    
    y = data['label'].values
    
    return pd.DataFrame(np.vstack([svd_dist, nmf_dist, wv1_dist, wv2_dist, ft_dist, y]).transpose(),
                       columns = ['svd_dist','nmf_dist', 'wv1_dist', 'wv2_dist', 'ft_dist', 'label'])

Получим необходимый нам датасет:

In [397]:
new_data = get_dataset(data, my_model.wv, ready_model, my_model1.wv)

Learning TF-IDF...
Learning SVD...
Learning NMF...
Calculating SVD distance...
Calculating NMF distance...


c:\users\k1l77\desktop\182e~1\term_p~1\lib\site-packages\scipy\spatial\distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Calculating Word2Vec distance1...
Calculating Word2Vec distance2...
Calculating FastText distance...


In [367]:
new_data.head()

,svd_dist,nmf_dist,wv1_dist,wv2_dist,ft_dist,label
0,0.566086,0.972402,0.091724,0.361776,0.082695,0.0
1,0.201763,0.001256,0.123796,0.179390,0.106179,0.0
2,0.263054,0.743416,0.048263,0.144536,0.144733,0.0
3,0.367196,0.392257,0.126012,0.533401,0.290044,-1.0
4,0.197857,0.141857,0.029121,0.299286,0.315807,0.0


In [398]:
new_data.shape

(7227, 6)

In [399]:
new_data.dropna().shape

(7217, 6)

In [400]:
new_data_clear = new_data.dropna()

Проведём классификацию и оценим качество:

In [385]:
clf = LogisticRegression(solver='liblinear')

In [386]:
X, y = new_data_clear.drop(['label'], axis=1), new_data_clear['label']

In [387]:
cross_val_scores = cross_val_score(clf, X, y, scoring=f1_micro)

In [388]:
cross_val_scores

array([0.56786704, 0.56371191, 0.6022176 , 0.48579349, 0.49410949])

In [389]:
np.mean(cross_val_scores)

0.5427399058978006

Средння f1 уже выше чем при предыдущем методе

<b>Вывод</b>: Использование расстояний между векторами текстов вместо самих векторов позволяет эффективнее решить задачу определения перефразирования

Попробуем увеличить количество измерений для NMF и SVD

In [396]:
tfidfmatrix = tfidf_vec.transform(all_texts)

nmf = NMF(100).fit(tfidfmatrix)
svd = TruncatedSVD(250).fit(tfidfmatrix)

nmf_transform = lambda x: nmf.transform(tfidf_vec.transform(x))
svd_transform = lambda x: svd.transform(tfidf_vec.transform(x))

In [401]:
new_data['nmf_dist'] = cosine_dist(data, nmf_transform)
new_data['svd_dist'] = cosine_dist(data, svd_transform)

c:\users\k1l77\desktop\182e~1\term_p~1\lib\site-packages\scipy\spatial\distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [402]:
X, y = new_data_clear.drop(['label'], axis=1), new_data_clear['label']

In [403]:
X.shape

(7217, 5)

In [404]:
clf = LogisticRegression(solver='liblinear')

In [405]:
X, y = new_data_clear.drop(['label'], axis=1), new_data_clear['label']

In [406]:
cross_val_scores = cross_val_score(clf, X, y, scoring=f1_micro)

In [407]:
cross_val_scores

array([0.57479224, 0.56440443, 0.6042966 , 0.48371448, 0.4961885 ])

Посмотрим на изменившуюся f-1:

In [408]:
np.mean(cross_val_scores)

0.5446792520199721

<b>Вывод</b>: Увеличив количество измерений в SVD и NMF, удалось увеличить качество классификации